# Load Zarr Image with labels from a public S3 repository, analyze using StarDist and compare results

[![Open in Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/ome/EMBL-EBI-imaging-course-05-2023/blob/main/Day_5/stardist.ipynb)


The notebook shows how to load an IDR image converted into a Zarr file with labels.

## Summary:
![Overview](./includes/StarDistNgff.png)

The image is referenced in the paper "NesSys: a novel method for accurate nuclear segmentation in 3D" published August 2019 in PLOS Biology: https://doi.org/10.1371/journal.pbio.3000388 and can be viewed online in the [Image Data Resource](https://idr.openmicroscopy.org/webclient/?show=image-6001247).

This original image was converted into the Zarr format. The analysis results produced by the authors of the paper were converted into labels and linked to the Zarr file which was placed into a public S3 repository.

In this notebook, the Zarr file is then loaded together with the labels from the S3 storage and analyzed using [StarDist](https://github.com/stardist/stardist). The StarDist analysis produces a segmentation, which is then viewed side-by-side with the original segmentations produced by the authors of the paper obtained via the loaded labels.



## Launch

This notebook uses the [environment_stardist.yml](./environment_stardist.yml) file.

See [Setup](./workshop.ipynb).


### Install dependencies if required

The cell below will install dependencies if you choose to run the notebook in [Google Colab](https://colab.research.google.com/notebooks/intro.ipynb#recent=true). **Do not run the cell if you are not running the notebook on Google Colab**.

If using Google Colab, **do not** use the ``Runtime>Run all`` entry.

In [ ]:
# Package to access data on S3
%pip install aiohttp==3.8.4 zarr==2.14.2

%pip install stardist==0.8.2

### Import 

In [1]:
import dask
import dask.array as da

import matplotlib.pyplot as plt
%matplotlib inline
from dask.diagnostics import ProgressBar
import numpy
import zarr

The Zarr data is stored separately from the IDR, on an S3 object store.

### Name of the file to analyze
The name of the file corresponds to the IDR identifier.

In [2]:
image_id = 6001247

In [3]:
ENDPOINT_URL = 'https://uk1s3.embassy.ebi.ac.uk/'

### Helper methods to load Zarr Image and Labels from S3

Stardist required the 5D image to be loaded.

In [5]:
def load_binary_from_s3(name, resolution='0'):
    root = '%s/%s/' % (name, resolution)
    with ProgressBar():
        return numpy.asarray(da.from_zarr(ENDPOINT_URL + root))

## Load the binary image from S3 

In [6]:
name = 'idr/zarr/v0.1/%s.zarr' % image_id
data = load_binary_from_s3(name)
print(data.shape)

[########################################] | 100% Completed | 21.45 s
(1, 2, 257, 210, 253)


## Load the labels
Labels are not on all planes, binary will be loaded when viewing the labels.

In [8]:
name = 'idr/zarr/v0.1/%s.zarr/labels' % image_id
labels = load_binary_from_s3(name)
print(labels.shape)

[########################################] | 100% Completed | 4.14 sms
(1, 1, 257, 210, 253)


## Load stardist trained model 

In [9]:
from stardist.models import StarDist2D
model_versatile = StarDist2D.from_pretrained('2D_demo')

2023-05-28 20:53:44.515580: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: SSE4.1 SSE4.2, in other operations, rebuild TensorFlow with the appropriate compiler flags.


Found model '2D_demo' for 'StarDist2D'.
Loading network weights from 'weights_best.h5'.
Loading thresholds from 'thresholds.json'.
Using default values: prob_thresh=0.486166, nms_thresh=0.5.


## Prediction based on a default StarDist model
Normalize the input image

``model.predict_instances`` will

 * predict object probabilities and star-convex polygon distances (see model.predict if you want those)
 * perform non-maximum suppression (with overlap threshold nms_thresh) for polygons above object probability threshold prob_thresh.
 * render all remaining polygon instances in a label image
 * return the label instances image and also the details (coordinates, etc.) of all remaining polygons

In [10]:
from csbdeep.utils import normalize
axis_norm = (0,1)
img = normalize(data[0, 1, :, :, :], 1,99.8, axis=axis_norm)
results = []
for im in img:
    new_labels, details = model_versatile.predict_instances(im)
    results.append(new_labels)

label_slices = numpy.array(results)

1/1 [==============================] - 0s 54ms/step


## Compare labels
Display the original labels and the labels based on StarDist prediction side-by-side

In [11]:
from ipywidgets import *

def update(z=0):
    fig = plt.figure(figsize=(10, 10))
    plt.subplot(121)
    c = 1
    plt.imshow(data[0, c, z, :, :], cmap='gray')
    try:
        plt.imshow(labels[0, c, z, :, :], cmap='jet', alpha=0.5)
    except Exception:
        print(z)
    plt.subplot(122)
    plt.imshow(data[0, c, z, :, :], cmap='gray')
    plt.imshow(label_slices[z, :, :], cmap='jet', alpha=0.5)
    plt.tight_layout()
    fig.canvas.flush_events()

interact(update, z= widgets.IntSlider(value=0, min=0, max=data.shape[2]-1, step=1, description="Select Z", continuous_update=False))

interactive(children=(IntSlider(value=0, continuous_update=False, description='Select Z', max=256), Output()),…

<function __main__.update(z=0)>

### License (BSD 2-Clause)
Copyright (C) 2023 University of Dundee. All Rights Reserved.

Redistribution and use in source and binary forms, with or without modification, are permitted provided that the following conditions are met:

Redistributions of source code must retain the above copyright notice, this list of conditions and the following disclaimer.
Redistributions in binary form must reproduce the above copyright notice, this list of conditions and the following disclaimer in the documentation and/or other materials provided with the distribution.
THIS SOFTWARE IS PROVIDED BY THE COPYRIGHT HOLDERS AND CONTRIBUTORS "AS IS" AND ANY EXPRESS OR IMPLIED WARRANTIES, INCLUDING, BUT NOT LIMITED TO, THE IMPLIED WARRANTIES OF MERCHANTABILITY AND FITNESS FOR A PARTICULAR PURPOSE ARE DISCLAIMED. IN NO EVENT SHALL THE COPYRIGHT OWNER OR CONTRIBUTORS BE LIABLE FOR ANY DIRECT, INDIRECT, INCIDENTAL, SPECIAL, EXEMPLARY, OR CONSEQUENTIAL DAMAGES (INCLUDING, BUT NOT LIMITED TO, PROCUREMENT OF SUBSTITUTE GOODS OR SERVICES; LOSS OF USE, DATA, OR PROFITS; OR BUSINESS INTERRUPTION) HOWEVER CAUSED AND ON ANY THEORY OF LIABILITY, WHETHER IN CONTRACT, STRICT LIABILITY, OR TORT (INCLUDING NEGLIGENCE OR OTHERWISE) ARISING IN ANY WAY OUT OF THE USE OF THIS SOFTWARE, EVEN IF ADVISED OF THE POSSIBILITY OF SUCH DAMAGE.